## `rocky`

In [1]:
import sys
# mydir = r'C:\Users\aweaver\rocky\rocky-app\src\py'
mydir = './src'
sys.path.append(mydir)
from rocky.rockycore import rocky
from rocky.triangle import Triangle

import pandas as pd
import numpy as np

In [2]:
# creat a new rocky object
r = rocky()

# inspect object
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t=())


In [3]:
## Could read in sample data (taylor_ashe, mack_1994) to try it out
r.SampleTri('taylor_ashe')

## Could copy a triangle from a file to the clipboard and then read it in
# r.FromClipboard('paid_loss')

## for this demo, we'll just read in a triangle from a link ratio file

# triangle file path for WC 4Q2022
# path = r'O:\STAFFHQ\SYMDATA\Actuarial\Reserving Applications\IBNR Allocation\4Q2022 Analysis\Workers Compensation 4Q2022.xlsb'

# read in triangle from file
# r.FromExcel(path, 1, 'paid_loss', 'Gross Paid Ultimates', "a28:u48") # takes filepath, number of origin columns, name of triangle, sheet name, and range of triangle
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t="paid_loss")


In [4]:
# access triangle object
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
r.paid_loss.acc.dt.year

0    2001
1    2002
2    2003
3    2004
4    2005
5    2006
6    2007
7    2008
8    2009
9    2010
Name: accident_period, dtype: int32

In [6]:
r.AddModel('glm', 'glm', 'paid_loss')
r

rocky(id='rocky0', mod="glm", f=(), plot=(), t="paid_loss")

In [8]:
# access linear design matrix:
r.paid_loss.X_base.head(10)

,is_observed,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_0002,development_period_0003,development_period_0004,development_period_0005,development_period_0006,development_period_0007,development_period_0008,development_period_0009,development_period_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0


In [9]:
# quick ATA factor summary
r.paid_loss.ata_summary()

Age-to-Age Factors as of (months),1,2,3,4,5,6,7,8,9
2001,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2002,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
2003,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
2004,4.568,1.547,1.712,1.073,1.087,1.047,,,
2005,2.564,1.873,1.362,1.174,1.138,,,,
2006,3.366,1.636,1.369,1.236,,,,,
2007,2.923,1.878,1.439,,,,,,
2008,3.953,2.016,,,,,,,
2009,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [10]:
# Triangle data type actually has a robust chain-ladder implementation that can be accessed

# this isn't important for right now, but it is there and it works well:
r.paid_loss.ult('vwa', 5, tail=1.061)

Accident Period
2001-01-01    4139452.0
2002-01-01    5765176.0
2003-01-01    5706935.0
2004-01-01    5621078.0
2005-01-01    5154550.0
2006-01-01    5422953.0
2007-01-01    5961413.0
2008-01-01    7197996.0
2009-01-01    6120629.0
2010-01-01    5011529.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [11]:
# now add a GLM model to the triangle:
r.AddModel('icrfs', 'loglinear', 'paid_loss')
print(r)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.
rocky(id='rocky0', mod=("glm", "icrfs"), f=(), plot=(), t="paid_loss")


In [20]:
r.icrfs.GetY('train')

0      357848.0
1      352118.0
2      290507.0
3      310608.0
4      443160.0
5      396132.0
6      440832.0
7      359480.0
8      376686.0
9      344014.0
10     766940.0
11     884021.0
12    1001799.0
13    1108250.0
14     693190.0
15     937085.0
16     847631.0
17    1061648.0
18     986608.0
20     610542.0
21     933894.0
22     926219.0
23     776189.0
24     991983.0
25     847498.0
26    1131398.0
27    1443370.0
30     482940.0
31    1183289.0
32    1016654.0
33    1562400.0
34     769488.0
35     805037.0
36    1063269.0
40     527326.0
41     445745.0
42     750816.0
43     272482.0
44     504851.0
45     705960.0
50     574398.0
51     320996.0
52     146923.0
53     352053.0
54     470639.0
60     146342.0
61     527804.0
62     495992.0
63     206286.0
70     139950.0
71     266172.0
72     280405.0
80     227229.0
81     425046.0
90      67948.0
Name: paid_loss, dtype: float64

In [12]:
r.icrfs.TuneFitHyperparameters()

Tuning on 5 and earlier: 100%|██████████| 651/651 [00:03<00:00, 165.29it/s]
/home/aweaver/git/rocky/rocky-app/./src/rocky/LogLinear.py:329: RuntimeWarning: overflow encountered in exp
  yhat = np.exp(self.model.predict(X))


In [19]:
dir(r.icrfs)

['Combine',
 'Deviance',
 'DevianceResiduals',
 'Fit',
 'GetAcc',
 'GetAccGp',
 'GetAccGpMap',
 'GetCal',
 'GetCalGp',
 'GetCalGpMap',
 'GetDegreesOfFreedom',
 'GetDev',
 'GetDevGp',
 'GetDevGpMap',
 'GetExposure',
 'GetHeteroGp',
 'GetIdx',
 'GetN',
 'GetP',
 'GetParameterNames',
 'GetParameters',
 'GetSaturatedModel',
 'GetVarY',
 'GetWeights',
 'GetX',
 'GetXBase',
 'GetY',
 'GetYBase',
 'GetYearTypeDict',
 'GetYhat',
 'LogLikelihood',
 'LogPi',
 'ManualFit',
 'PearsonResiduals',
 'Predict',
 'PredictTriangle',
 'RawResiduals',
 'ScaleParameter',
 'SetCalGpMap',
 'SetDevGpMap',
 'SetHeteroGp',
 'SetHyperparameters',
 'SetVarY',
 'SetWeights',
 'SetX',
 'SetY',
 'TuneFitHyperparameters',
 'Ultimate',
 'X_forecast',
 'X_train',
 '_CalcHeteroAdj',
 '_FitData',
 '_ProcessVarMLE',
 '_ProcessVarUBE',
 '_StandardError',
 '_StandardizedResiduals',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__

In [ ]:
r.glm.TuneFitHyperparameters()

In [ ]:
# load hyperparameters
# r.tweedie.TuneFitHyperparameters()
r.icrfs.SetHyperparameters(alpha=0.5, l1_ratio=0.5)
# r.tweedie.SetHyperparameters(alpha=0.2, power=2.3)
# r.tweedie.SetHyperparameters(alpha=1, power=1)
r.icrfs

loglinear(alpha=0.5, l1_ratio=0.5)

In [ ]:
r.icrfs.GetY('train')
# r.icrfs.is_fitted

0      357848.0
1      352118.0
2      290507.0
3      310608.0
4      443160.0
5      396132.0
6      440832.0
7      359480.0
8      376686.0
9      344014.0
10     766940.0
11     884021.0
12    1001799.0
13    1108250.0
14     693190.0
15     937085.0
16     847631.0
17    1061648.0
18     986608.0
20     610542.0
21     933894.0
22     926219.0
23     776189.0
24     991983.0
25     847498.0
26    1131398.0
27    1443370.0
30     482940.0
31    1183289.0
32    1016654.0
33    1562400.0
34     769488.0
35     805037.0
36    1063269.0
40     527326.0
41     445745.0
42     750816.0
43     272482.0
44     504851.0
45     705960.0
50     574398.0
51     320996.0
52     146923.0
53     352053.0
54     470639.0
60     146342.0
61     527804.0
62     495992.0
63     206286.0
70     139950.0
71     266172.0
72     280405.0
80     227229.0
81     425046.0
90      67948.0
Name: paid_loss, dtype: float64

In [ ]:
## now that we have a set of hyperparameters, we can reuse them
## next time using model.SetHyperparameters(alpha, power):
# -> r.tweedie.SetHyperparameters(alpha=0.4, power=1.8)

# fit base model to the data
r.icrfs.Fit()
print(r)

rocky(id='rocky0', mod="icrfs", f=(), plot=(), t="paid_loss")


/home/aweaver/git/rocky/rocky-app/./src/rocky/LogLinear.py:329: RuntimeWarning: overflow encountered in exp
  yhat = np.exp(self.model.predict(X))


In [ ]:
r.icrfs.model

ElasticNet(alpha=0.5, fit_intercept=False, max_iter=100000)

In [ ]:
r.icrfs.model.coef_

array([ 183378.14553397,  140318.94833352,   69085.79388944,
         42868.98474404,   29947.69303848,   44827.05366262,
         46096.02812581,   22861.82518928,  -15757.27538682,
        -14888.09811053,  247990.79104194,   99322.85216628,
         -5048.99606003, -122806.4928787 ,  -92180.66561062,
        -50545.17421981,  -31198.61043775,   -5748.99327737,
        -10522.10889884])

In [ ]:
r.icrfs.PredictTriangle().pivot_table(index='accident_period', columns='development_period', values='yhat').round(0)

/home/aweaver/git/rocky/rocky-app/./src/rocky/LogLinear.py:329: RuntimeWarning: overflow encountered in exp
  yhat = np.exp(self.model.predict(X))


development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2002,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2003,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2004,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2005,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2006,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2007,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2008,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2009,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [ ]:
r.paid_loss.incr_triangle.astype(float).round(0)

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,357848.0,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2002,352118.0,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
2003,290507.0,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
2004,310608.0,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
2005,443160.0,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
2006,396132.0,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
2007,440832.0,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
2008,359480.0,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,376686.0,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.icrfs.train_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36,
       40, 41, 42, 43, 44, 45, 50, 51, 52, 53, 54, 60, 61, 62, 63, 70, 71,
       72, 80, 81, 90])

In [ ]:
r.icrfs.PearsonResiduals()

/home/aweaver/git/rocky/rocky-app/./src/rocky/LogLinear.py:329: RuntimeWarning: overflow encountered in exp
  yhat = np.exp(self.model.predict(X))


AttributeError: 'LogLinear' object has no attribute 'VarY'

In [ ]:
# chain ladder ult vs GLM
df = pd.concat([pd.DataFrame(r.paid_loss.ult()), pd.DataFrame(r.icrfs.Ultimate())], axis=1)
df = df.assign(Diff=df['Log-linear Ultimate'] - df['Chain Ladder Ultimate Loss']).astype(int)
df['Diagonal'] = r.paid_loss.diag().astype(int)
df['Unpaid'] = df['Log-linear Ultimate'] - df.Diagonal
df

: 

In [ ]:
r.icrfs.plot.obs_pred(log=True, color='dev')

: 

In [ ]:
r.tweedie.plot.residual('acc', log=True)

: 

In [ ]:
r.tweedie.plot.residual('dev', log=False)

: 

In [ ]:
r.paid_loss.tri

: 

In [ ]:
## accident period parameters should roughly follow the first column:
r.paid_loss.tri.iloc[:, 0].plot()

: 

In [ ]:
# but the actual fitted parameters include regularization terms that smooth out some of these bumps
r.tweedie.PlotParameter('acc')

: 

In [ ]:
# does not include dev_period_0012 - first development period is assumed to be included in the first accident period parameter
r.tweedie.PlotParameter('dev')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
# final 2 accident years don't change much from 2020 - can set parameters for these years to be 0 (for no change)
r.tweedie.ManualFit(
  accident_period_2021=0, accident_period_2022=0,
  accident_period_2005=0,
  accident_period_2013=0,
  accident_period_2016=0,
  accident_period_2018=0
  )

: 

In [ ]:
r.tweedie.PlotParameter('acc')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
## TODO
# - add a method to do a future calendar year projection
# - add a method to combine parameters into a single parameter for each year

: 

In [ ]:
# add a second glm model that includes calendar year effects
r.AddModel('tweediecal', 'glm', 'paid_loss', cal=True)
print(r)

: 

In [ ]:
# can look at design matrix for this model -- it has 39 more columns for the 39 calendar periods in a 20x20 triangle
r.tweediecal.GetX()

: 

In [ ]:
# using same hyperparameters as before, but could decide to refit
r.tweediecal.SetHyperparameters(alpha=0.3, power=1.8)
r.tweediecal

: 

In [ ]:
r.tweediecal.Fit()

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
r.tweediecal.PlotParameter('acc')

: 

In [ ]:
r.tweediecal.PlotParameter('dev')

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
## add a 1% future cal trend (eg assume that trend reverses itself)
future_cy_trend = 0.01
manfit = {}
for i in range(21, 40):
  manfit[f"calendar_period_00{i}"] = future_cy_trend

r.tweediecal.ManualFit(**manfit)

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
def parse_param_name(x):
  if x.find("_") == -1:
    return x
  else:
    end = x.split("_")[len(x.split("_"))-1]
    out = x.replace(f"_{end}", "")[:3]
    return out
  

df = pd.DataFrame(dict(nm=r.tweediecal.GetParameterNames(), val=r.tweediecal.model.coef_))
parse_param_name(df.nm[2])

: 

In [ ]:
df = pd.DataFrame({
  'chain ladder': r.paid_loss.ult(tail=1.05),
  'tweedie': r.tweedie.Ultimate(tail=1.05),
  'tweediecal': r.tweediecal.Ultimate(tail=1.05)
}).round(-3)/1000

df

: 

In [ ]:
r.Add

: 